# 波士顿房价预测案例——线性回归分析

比较对y和logy进行线性拟合的效果。

## 1、导入必要的工具包

In [1]:
import numpy as np  # 矩阵操作
import pandas as pd # SQL数据处理

from sklearn.metrics import r2_score  #评价回归预测模型的性能

## 2. 读取数据
已经是做完特征工程后的数据，请先运行2_FE_BostonHousePrice.ipynb，得到文件FE_boston_housing.csv

In [2]:
# path to where the data lies
#dpath = './data/'
df = pd.read_csv("FE_boston_housing.csv")

#通过观察前5行，了解数据每列（特征）的概况
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,TAX,PTRATIO,...,RAD_2,RAD_3,RAD_4,RAD_5,RAD_6,RAD_7,RAD_8,RAD_24,MEDV,log_MEDV
0,-0.419782,0.285654,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.666608,-1.353192,...,0,0,0,0,0,0,0,0,0.159686,0.345176
1,-0.417339,-0.487292,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.987329,-0.475352,...,1,0,0,0,0,0,0,0,-0.101524,0.084104
2,-0.417342,-0.487292,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.987329,-0.475352,...,1,0,0,0,0,0,0,0,1.324247,1.266776
3,-0.416750,-0.487292,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-1.106115,-0.036432,...,0,1,0,0,0,0,0,0,1.182758,1.170822
4,-0.412482,-0.487292,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-1.106115,-0.036432,...,0,1,0,0,0,0,0,0,1.487503,1.373242


###  数据基本信息
样本数目、特征维数
每个特征的类型、空值样本的数目、数据类型

In [3]:
#df.info()

### 数据准备

In [4]:
# 从原始数据中分离输入特征x和输出y
# 这里我们y有2个取值，原始的MEDV及其log1p之后的值
col_y = ["MEDV","log_MEDV"]
y = pd.DataFrame(df,columns = col_y)

X = df.drop(["MEDV", "log_MEDV"], axis = 1)

#特征名称，用于后续显示权重系数对应的特征
feat_names = X.columns

当数据量比较大时，可用train_test_split从训练集中分出一部分做校验集；
样本数目较少时，建议用交叉验证。
在线性回归中，留一交叉验证有简便计算方式。

下面将训练数据分割成训练集和测试集，只是让大家对模型的训练误差、校验集上的测试误差估计、和测试集上的测试误差做个比较。

In [5]:
#将数据分割训练数据与测试数据
from sklearn.model_selection import train_test_split

# 随机采样20%的数据构建测试样本，其余作为训练样本
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33, test_size=0.2)
X_train.shape

(404, 21)

## 3、确定模型类型

### 3.1 尝试缺省参数的线性回归

In [6]:
# 线性回归
#class sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
from sklearn.linear_model import LinearRegression

# 1.使用默认配置初始化学习器实例
lr = LinearRegression()

# 2.用训练数据训练模型参数
lr.fit(X_train, y_train)

# 3. 用训练好的模型对测试集进行预测
y_test_pred_lr = lr.predict(X_test)
y_train_pred_lr = lr.predict(X_train)


# 看看各特征的权重系数，系数的绝对值大小可视为该特征的重要性
fs = pd.DataFrame({"columns":list(feat_names), "coef_org":list((lr.coef_[0,:].T)),"coef_log":list((lr.coef_[1,:].T))})
fs.sort_values(by=['coef_org'],ascending=False)

,coef_log,coef_org,columns
20,0.524147,0.505543,RAD_24
5,0.176170,0.297984,RM
18,0.157229,0.184468,RAD_7
1,0.094180,0.147403,ZN
19,0.093588,0.147279,RAD_8
14,0.058141,0.134516,RAD_3
10,0.100203,0.088869,B
3,0.071025,0.074143,CHAS
2,0.048219,0.017002,INDUS
6,0.004709,-0.001742,AGE


#### 3.1.1 模型评价

In [7]:
# 使用r2_score评价模型在测试集和训练集上的性能，并输出评估结果
#测试集
print 'The r2 score of LinearRegression on test with original MEDV is', r2_score(y_test.iloc[:,0], y_test_pred_lr[:,0])
#训练集
print 'The r2 score of LinearRegression on train with original MEDV is', r2_score(y_train.iloc[:,0], y_train_pred_lr[:,0])

# y取log
#测试集
print 'The r2 score of LinearRegression on test with log MEDV is', r2_score(y_test.iloc[:,1], y_test_pred_lr[:,1])
#训练集
print 'The r2 score of LinearRegression on train  with log MEDV is', r2_score(y_train.iloc[:,01], y_train_pred_lr[:,1])

The r2 score of LinearRegression on test with original MEDV is 0.693978981051
The r2 score of LinearRegression on train with original MEDV is 0.754914643687
The r2 score of LinearRegression on test with log MEDV is 0.708305459026
The r2 score of LinearRegression on train  with log MEDV is 0.811201041743


对y（价格）取log后，r2 score略变好。